In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install flask==0.12.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.3 MB/s 
  Attempting uninstall: flask
    Found existing installation: Flask 1.1.4
    Uninstalling Flask-1.1.4:
      Successfully uninstalled Flask-1.1.4


In [ ]:
!pip install pyngrok==4.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15983 sha256=f7e6143da07b74356ff9494133394faa75a52b15e68373a059ccfb2e47d24b6b
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok


In [ ]:
from __future__ import division, print_function
# Flask utils
from flask_ngrok import run_with_ngrok
from flask import Flask, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten,Input,add
from tensorflow.keras.layers import Conv2D,Add,ZeroPadding2D,AveragePooling2D
from tensorflow.keras.layers import MaxPooling2D,GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from keras.losses import categorical_crossentropy
import os

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

In [ ]:
!pip install pyngrok==4.1.1
!ngrok authtoken 'your authtoken'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15983 sha256=c0863448c070963042bb90325f35b252d2a4afb74e335ae7131b33f667f234fa
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
%cd /content/drive/MyDrive/MTech \AI \NUS/PRACTICE \MODULE/

/content/drive/MyDrive/MTech AI NUS/PRACTICE MODULE


In [ ]:
os.getcwd()

'/content/drive/MyDrive/MTech AI NUS/PRACTICE MODULE'

In [ ]:
def read_image(path):
    file_bytes = tf.io.read_file(path)
    img = tf.image.decode_png(file_bytes, channels=3)
    img = tf.cast(img, tf.float32) / 255.0
    img = tf.image.resize(img, [200,200], method="nearest")
    return img

In [ ]:
def resIdentity_blk(x, filter):
    x_skip = x
    
    #Layer 1
    x = Conv2D(filter, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    #Layer 2
    x = Conv2D(filter, (3,3), padding='same')(x)
    x = BatchNormalization()(x)

    #Add Layer
    x = Add()([x, x_skip])
    x = Activation('relu')(x)

    return x

def resConv_blp(x, filter):
    x_skip = x

    #Layer 1
    x = Conv2D(filter, (3,3), padding='same', strides = (2,2))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    #Layer 2
    x = Conv2D(filter, (3,3), padding='same')(x)
    x = BatchNormalization()(x)

    #Skip Layer
    x_skip = Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    #x_skip = BatchNormalization()(x_skip)

    #Add Layer
    x = Add()([x, x_skip])
    x = Activation('relu')(x)

    return x

def build_residualCNN(num_classes=3):
  input = tf.keras.Input(shape=(200, 200, 3), name='img')
  #x = ZeroPadding2D((3,3))(inputs)

  #Beginning Conv Layer with MaxPool
  x = Conv2D(16, (3,3), padding='valid', strides=(2,2))(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  #x = MaxPooling2D(pool_size=(3,3), strides=(2,2))(x)

  x = resConv_blp(x,16)
  x = resIdentity_blk(x,16)
  x = resIdentity_blk(x,16)

  x = resConv_blp(x,32)
  x = resIdentity_blk(x,32)
  x = resIdentity_blk(x,32) 

  x= AveragePooling2D(pool_size=(8,8))(x)
  x= Flatten()(x)

  output = Dense(num_classes, activation='softmax',kernel_initializer='he_normal')(x)
  model = Model(inputs = input,outputs = output)
  return model

In [ ]:
class prediction():
    
    def __init__(self,classify_model_weights,num_classes=3):
    
        self.final_classify_model = build_residualCNN()
    
        self.final_classify_model.load_weights(classify_model_weights)
        self.final_classify_model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.001), metrics=['acc'])
        print('Residual CNN Model Loaded....')
    
    def Predict(self,image_path):
        """
        This function predicts the classfication output.
        ------------------------------------------------------------------
        final_classify_model      : saved classification model instance
        image_path                : image path
        ------------------------------------------------------------------
        """

         # read the original image
        image_orig = read_image(image_path)
    
        #get the classification output
        classify_output = self.final_classify_model.predict(tf.expand_dims(image_orig, axis=0))
        print(classify_output)

In [ ]:
classify_weights_path = "/content/drive/MyDrive/MTech AI NUS/PRACTICE MODULE/resnetModel17.h5"
pred = prediction(classify_weights_path)

Residual CNN Model Loaded....


In [ ]:
classify_output = pred.Predict('contaminated_food_container_15.jpeg')

1/1 [==============================] - 0s 38ms/step
[[9.9896705e-01 2.1199542e-04 8.2099112e-04]]


'plastic_bowl_trash_538.jpeg' -- dirty (true) dirty (predict)

'plastic_bowl_trash_42.jpeg' -- clean (true) clean (predict)

'0ace9ea7880444b799a6c5e2d0154147.jpeg' -- non (true) clean (predict)

'aerosol_can_22.jpeg' -- clean (true) clean (predict)

'contaminated_food_container_15' -- dirty (true) non (predict)





In [ ]:
#!ngrok authtoken sma1234

# flask_ngrok_example.py
from flask import Flask
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/")
def hello():
    return "Hello World!"

if __name__ == '__main__':
   app.run()

INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
Exception in thread Thread-15:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connection.py", line 159, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/usr/local/lib/python3.7/dist-packages/urllib3/util/connection.py", line 80, in create_connection
    raise err
  File "/usr/local/lib/python3.7/dist-packages/urllib3/util/connection.py", line 70, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 354, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/usr/lib/python3.7/ht